In [1]:
import pandas as pd
import numpy as np

In [2]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs


from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

# Reading in Data

In [15]:
# load in params and websites
# webs_paras = pd.read_csv('../data/nor_time.csv')
webs = webs_paras.drop(['(Doc complete) Byets in','(Fully loaded) Bytes in','(Fully loaded) Requests','(Fully loaded) Requests','(Doc complete) Requests'], axis = 1)
#webs_requests = webs_paras[['Site name','(Fully loaded) Requests','(Fully loaded) Requests','(Doc complete) Requests']]



 #webs['(Doc complete) Byets in'].describe()

list_of_params = list(webs_paras.columns[1:].unique())

list_of_params.sort()

#len(list_of_params) 

#for i in range(49):
    #print i%10
# webs.min()
#webs_paras['(Fully loaded) Bytes in'].max()

ValueError: labels ['(Doc complete) Byets in' '(Fully loaded) Bytes in'
 '(Fully loaded) Requests' '(Fully loaded) Requests'
 '(Doc complete) Requests'] not contained in axis

In [16]:
webs_paras

,Unnamed: 0,(Fully loaded) Time,DOM elements,First byte,Load time,Speed Index,Start render
0,0,0.205970,0.032137,0.068536,0.195896,0.155050,0.232877
1,1,0.269239,0.269806,0.388024,0.212793,0.507788,0.315068
2,2,0.247995,0.382460,0.331256,0.170562,0.310337,0.232877
3,3,0.034321,0.111830,0.003115,0.036822,0.044210,0.068493
4,4,0.143604,0.488405,0.193493,0.172448,0.186989,0.219178
5,5,0.688190,0.070277,0.220492,0.807106,0.239223,0.410959
6,6,0.429321,0.159741,0.105573,0.120763,0.219792,0.246575
7,7,0.165960,0.280636,0.062305,0.196144,0.182347,0.273973
8,8,0.246253,0.054974,0.496366,0.287430,0.655365,1.000000
9,9,0.109660,0.043320,0.061613,0.018609,0.043974,0.013699


# Function to Make Dataset for plot


In [17]:
def make_dataset(params_list, range_start = 0, range_end = 1, bin_width =0.005):
    
    #check to make sure the start is less than the end
    assert range_start < range_end, "Start must be less than end!"
    
    #by_params = pd.DataFrame(columns=[ ,'Max', 'Avarage', 'Min','color'])
    by_params = pd.DataFrame(columns=[ 'left','right', 'proportion', 'p_proportion','p_interval', 'name', 'color']) 
   
    
    range_extent = range_end - range_start
    values = ['Min', "Avarage", 'Max']
    # Iterate through all the parameters 
    for i, para_name in enumerate(params_list):
        
        #print para_name
        # Subset to the parameter
        subset = webs_paras[para_name]
        
        print(webs_paras)
       
        # note: subset have to be a list of values
        
        # [webs.columns[i%6]]
        
        # Create a histogram with specified bins and range
        arr_hist, edges = np.histogram(subset,  
                                      bins = int(range_extent / bin_width),
                                      range = [range_start, range_end])

        # Divide the counts by the total to get a proportion and create df
        arr_df= pd.DataFrame({'proportion': arr_hist ,
                              'left': edges[:-1], 'right': edges[1:]}) #/ np.sum(arr_hist)
      
        # Format the proportion
        arr_df['p_proportion'] = ['%0.00005f' % proportion for proportion in arr_df['proportion']]
        
        # Format the interval
        arr_df['p_interval'] = ['%d to %d scale' % (left, right) for left, 
                               right in zip(arr_df['left'], arr_df['right'])]
        
        # Assign the parameter for labels
        arr_df['name'] = para_name
        arr_df['w_name'] = webs['Site name']
        
        # Color each parametr differently
        arr_df['color'] = Category20_16[i]
        
        # Add to the overall dataframe
        by_params = by_params.append(arr_df)
        
    # Overall dataframe
    by_params = by_params.sort_values(['name','left'])
    
    
    
    return by_params
        
        
        
        

In [18]:
make_dataset(list_of_params)

    Unnamed: 0  (Fully loaded) Time  DOM elements  First byte  Load time  \
0            0             0.205970      0.032137    0.068536   0.195896   
1            1             0.269239      0.269806    0.388024   0.212793   
2            2             0.247995      0.382460    0.331256   0.170562   
3            3             0.034321      0.111830    0.003115   0.036822   
4            4             0.143604      0.488405    0.193493   0.172448   
5            5             0.688190      0.070277    0.220492   0.807106   
6            6             0.429321      0.159741    0.105573   0.120763   
7            7             0.165960      0.280636    0.062305   0.196144   
8            8             0.246253      0.054974    0.496366   0.287430   
9            9             0.109660      0.043320    0.061613   0.018609   
10          10             0.060498      0.044732    0.126341   0.029254   
11          11             0.230803      0.171748    0.103150   0.153417   
12          

NameError: global name 'webs' is not defined

# Function to Make Plot from Data Source

In [221]:
def style(p):
    # Title
    p.title.align = 'center'
    p.title.text_font_size ='20pt'
    p.title.text_font = 'serif'
    
    # Axis titles
    p.xaxis.axis_label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '14pt'
    p.yaxis.axis_label_text_font_style = 'bold'
    
    # Tick labels
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'
    
    return p

In [222]:
def make_plot(src):
    # Blank plot with correct labels
    p = figure(plot_width = 700, plot_height = 700,
              title = "Histogram of Parametes For The Websites",
              x_axis_label = 'Parameters', y_axis_label = "Values")
    
    # Quad glyphs to create a histogram
    p.quad(source=src, bottom =0,left = 'left', right = 'right', color ='color', top= 'proportion',fill_alpha = 0.7, hover_fill_color = 'color', legend = 'name',
           hover_fill_alpha = 1.0, line_color = 'white') #top='proportion',
    
    # Hover tool with vline mode
    hover = HoverTool(tooltips=[('Website','@w_name'),
                                ('Parameter','@name'),
                                ('Proportion','@p_proportion')
                               ],
                     mode='vline')
    
    p.add_tools(hover)
    # Stypling
    p = style(p)
    return p

In [223]:
p=make_plot(ColumnDataSource(make_dataset(list_of_params)))
show(p)

ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): ValueError("expected a dict or pandas.DataFrame, got ColumnDataSource(id='18a2296e-4e4e-4210-9e21-0d3c7b10454f', ...)",)
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): ValueError("expected a dict or pandas.DataFrame, got ColumnDataSource(id='6f3c5cb7-8557-45b5-bddf-c191007cdd2f', ...)",)
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): ValueError("expected a dict or pandas.DataFrame, got ColumnDataSource(id='4c974b0b-4d9f-4b7c-b6df-90736c372c0f', ...)",)
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): ValueError("expected a dict or pandas.DataFrame, got ColumnDataSource(id='e298f89a-7176-496e-be71-198b3a59c324', ...)",)
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): ValueError("expected a dict or pandas.DataFrame, go

# Checkbox Group Element for Selecting parameter

In [51]:
para_selection = CheckboxGroup(labels=list_of_params, active = [0,1])
show(para_selection)

In [52]:
[para_selection.labels[i] for i in para_selection.active]

['(Fully loaded) Time', 'DOM elements']

# Update Function for Parameter Selected

In [53]:
# Update function takes three default parameters
def update(attr, old, new):
    
    # Get the list of parameter for the graph
    parameter_to_plot = [para_selection.labels[i] for i in para_selection.active]
    
    # Make a new dataset based on the selected parameter and the 
    # make_dataset function defined earlier
    new_src = make_dataset(parameter_to_plot) # note range are not specified
    
    
    # Convert dataframe to column data source
    new_src = ColumnDataSource(new_src)
    
    # Update the source used the quad glpyhs
    src.data.update(new_src.data)

In [168]:
def modify_doc(doc):
    def make_dataset(params_list, range_start = 0.0, range_end = 1, bin_width = 0.005):
        
        #check to make sure the start is less than the end
        assert range_start < range_end, "Start must be less than end!"

        #by_params = pd.DataFrame(columns=[ ,'Max', 'Avarage', 'Min','color'])
        by_params = pd.DataFrame(columns=[ 'left','right', 'proportion', 'p_proportion','p_interval', 'name', 'color']) 
            # 

        range_extent = range_end - range_start
        values = ['Min', "Avarage", 'Max']
        # Iterate through all the parameters 
        for i, para_name in enumerate(params_list):

            #print para_name
            # Subset to the parameter
            subset = webs[para_name]

            # note: subset have to be a list of values

            # [webs.columns[i%6]]

            # Create a histogram with specified bins and range
            arr_hist, edges = np.histogram(subset,  
                                          bins = int(range_extent / bin_width),
                                          range = [range_start, range_end])

            # Divide the counts by the total to get a proportion and create df
            arr_df= pd.DataFrame({'proportion': arr_hist ,
                                  'left': edges[:-1], 'right': edges[1:]}) #/ np.sum(arr_hist)

            # Format the proportion
            arr_df['p_proportion'] = ['%0.00005f' % proportion for proportion in arr_df['proportion']]

            # Format the interval
            arr_df['p_interval'] = ['%d to %d scale' % (left, right) for left, 
                                   right in zip(arr_df['left'], arr_df['right'])]

            # Assign the parameter for labels
            arr_df['name'] = para_name

            # Color each parametr differently
            arr_df['color'] = Category20_16[i]
            #print arr_df

            # Add to the overall dataframe
            by_params = by_params.append(arr_df)

        # Overall dataframe
        by_params = by_params.sort_values(['name','left'])
    
        return ColumnDataSource(by_params)


    def style(p):
        # Title
        p.title.align = 'center'
        p.title.text_font_size ='20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p


    def make_plot(src):
        # Blank plot with correct labels
        p = figure(plot_width = 700, plot_height = 700,
                  title = "Histogram of Parametes For The Websites",
                  x_axis_label = 'x_label', y_axis_label = "y_labe")

        # Quad glyphs to create a histogram
        p.quad(source=src, bottom =0,left = 'left', right = 'right', color ='color', top= 'proportion',fill_alpha = 0.7, hover_fill_color = 'color', legend = 'name',
               hover_fill_alpha = 1.0, line_color = 'color') #top='proportion',

        # Hover tool with vline mode
        hover = HoverTool(tooltips=[('Parameter','@name'),
                                   ('Difference','@p_interval'),
                                    ('Proportion','p_proportion')
                                   ],
                         mode='vline')

        p.add_tools(hover)
        # Stypling
        p = style(p)
        return p
     
    # Update function takes three default parameters
    def update(attr, old, new):
        
        
        # Get the list of parameter for the graph
        parameter_to_plot = [para_selection.labels[i] for i in para_selection.active]
        
        

        # Make a new dataset based on the selected parameter and the 
        # make_dataset function defined earlier
        new_src = make_dataset(parameter_to_plot, 
                               range_start = range_select.value[0],
                               range_end = range_select.value[1],
                               bin_width = binwidth_select.value) # note range are not specified


        # Convert dataframe to column data source
        new_src = ColumnDataSource(new_src)

        # Update the source used the quad glpyhs
        src.data.update(new_src.data)

    list_of_params = list(webs.columns[1:].unique())
    list_of_params.sort()
    
    para_colors = Category20_16
    para_colors.sort()
    
    
    para_selection = CheckboxGroup(labels=list_of_params, active = [0,1])
    para_selection.on_change('active',update)
   
    binwidth_select = Slider(start =0, end = 1.5,
                            step = 0.00025, value = 0.0005,
                            title = 'Change in parameter')
    binwidth_select.on_change('value', update)
    
    range_select = RangeSlider(start=0, end=1, value =(0,1),
                             step=0.00025, title = 'Change in range')
    range_select.on_change('value', update)
    
    initial_params = [para_selection.labels[i] for i in para_selection.active]
    
    for i in initial_params:
        print i
    
    src = make_dataset(initial_params,
                      range_start = range_select.value[0],
                      range_end = range_select.value[1],
                      bin_width = binwidth_select.value)
    
    print "here"
    p = make_plot(src)
    #show(p)
    # Put controls in a single element
    controls = WidgetBox(para_selection, binwidth_select, range_select)
    
    # Create a row layout
    layout = row(controls, p)
    
    # Make a tab with the layout
    tab = Panel(child = layout, title = 'Histogram')
    tabs = Tabs(tabs=[tab])
    
    doc.add_root(tabs)
    
# Set up an application
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [169]:
#modify_doc(webs)

In [170]:
show(app, 'localhost:9000')

(Fully loaded) Time
DOM elements
here


ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): ValueError("expected a dict or pandas.DataFrame, got ColumnDataSource(id='9d5a098f-d79d-46db-b87d-071e9f5563b2', ...)",)
